# Prédiction de la demande pour le modèle d'optimisation
Lors de la séance 8, nous avons créé et entraîné un modèle prédictif de la demande par UPC. Maintenant, dans ce notebook, nous allons préparer nos entrées pour le modèle d'optimisation en prédisant la demande pour différents niveaux de prix.

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# Données d'entrée
Nous avons préparé les fichiers d'entrée qui contiennent les caractéristiques qui nous serviront à faire les prédictions. Le premier fichier contient un petit jeu de données tandis que le second fichier contient un plus gros jeu de données. En particulier,
1.   **'InputFeatures_Prob1.csv'**: C'est un problème à petite échelle. Les prédictions résultantes de ces données seront utilisées dans les modèles d'optimisation détaillé et compact.
2.   **'InputFeatures_Prob2.csv'**: C'est un problème à grande échelle. Les prédictions résultantes de ces données seront utilisées seulement dans le modèle compact.

*Pour faire les prédictions sur un jeu de données ou l'autre, vous devez définir `SIZE = 'small'` ou `SIZE = 'large'`.*

In [2]:
# Le paramètre SIZE doit être défini à 'small' ou 'large'
SIZE = 'small'

if SIZE == 'small':
    # petit exemple
    url = 'data/InputFeatures_Prob1.csv'
elif SIZE == 'large':
    # grand exemple
    url = 'data/InputFeatures_Prob2.csv'
else:
    raise ValueError("SIZE doit être défini à 'small' ou 'large'")

predDemand = pd.read_csv(filepath)
print(predDemand.shape)
predDemand

(12, 8)


,AVG_PRICE_VALUE,UPC,PRICE,PRICE_p2,FEATURE,DISPLAY,TPR_ONLY,RELPRICE
0,3.0,1600027528,2.5,6.25,0,0,0,0.833333
1,3.0,1600027528,3.0,9.00,0,0,0,1.000000
2,3.0,1600027528,3.5,12.25,0,0,0,1.166667
3,3.0,1600027564,2.5,6.25,0,0,0,0.833333
4,3.0,1600027564,3.0,9.00,0,0,0,1.000000
5,3.0,1600027564,3.5,12.25,0,0,0,1.166667
6,3.0,3000006340,2.5,6.25,0,0,0,0.833333
7,3.0,3000006340,3.0,9.00,0,0,0,1.000000
8,3.0,3000006340,3.5,12.25,0,0,0,1.166667
9,3.0,3800031829,2.5,6.25,0,0,0,0.833333


# Récupération de modèle
Dans la cellule suivante, nous récupérons les meilleurs modèles que nous avions obtenus à la séance précédente. Pour cela, nous entraînons les modèles sur les même jeux de données et obtenons ainsi les modèles entraînés.

In [3]:
salesCereals = pd.read_csv('data/salesCereals.csv')  # importe les données

feature_list = ['PRICE', 'PRICE_p2', 'FEATURE', 'DISPLAY', 'TPR_ONLY', 'RELPRICE']  # caractéristiques utilisées
productList = salesCereals['UPC'].unique()  # codes UPC

# Obtient les données d'entraînement par UPC,
# puis entraîne chacun des modèles
regr = {}
for upc in productList:  
    X_train = salesCereals.loc[salesCereals['UPC'] == upc, feature_list]
    y_train = salesCereals.loc[salesCereals['UPC'] == upc, 'UNITS']
    regr[upc] = LinearRegression().fit(X_train, y_train)

salesCereals.head()

,WEEK_END_DATE,STORE_NUM,UPC,UNITS,VISITS,HHS,SPEND,PRICE,BASE_PRICE,FEATURE,DISPLAY,TPR_ONLY,DESCRIPTION,CATEGORY,SUB_CATEGORY,SUMPRICE,COUNTPRICE,AVGPRICE,RELPRICE,PRICE_p2
0,2009-01-14,367,1111085319,14,13,13,26.32,1.88,1.88,0,0,0,PL HONEY NUT TOASTD OATS,COLD CEREAL,ALL FAMILY CEREAL,19.54,7,2.791429,0.673490,3.5344
1,2009-01-14,367,1111085350,35,27,25,69.30,1.98,1.98,0,0,0,PL BT SZ FRSTD SHRD WHT,COLD CEREAL,ALL FAMILY CEREAL,19.54,7,2.791429,0.709314,3.9204
2,2009-01-14,367,1600027527,12,10,10,38.28,3.19,3.19,0,0,0,GM HONEY NUT CHEERIOS,COLD CEREAL,ALL FAMILY CEREAL,19.54,7,2.791429,1.142784,10.1761
3,2009-01-14,367,1600027528,31,26,19,142.29,4.59,4.59,0,0,0,GM CHEERIOS,COLD CEREAL,ALL FAMILY CEREAL,19.54,7,2.791429,1.644319,21.0681
4,2009-01-14,367,1600027564,56,48,42,152.32,2.72,3.07,1,0,0,GM CHEERIOS,COLD CEREAL,ALL FAMILY CEREAL,19.54,7,2.791429,0.974411,7.3984


# Prévision de la demande
Dans cette cellule, nous exécutons également une boucle **pour** chaque UPC. Voici la descriptions de chaque ligne de la boucle:
* La première ligne de la boucle identifie les lignes dans `predDemand` qui correspondent à `upc`.
* La deuxième ligne récupère les caractéristiques de `upc`.
* La troisième ligne prend le modèle pour l'UPC courant (`regr[upc]`) et prédit la demande. Nous utilisons également la fonction `clip(min=0.)` pour s'assurer que la demande n'est pas négative et la fonction `round(decimals=1)` pour arrondir la valeur prédite à une décimale.
* La quatrième ligne place la demande prédite dans la colonne `'PRED_SALES'` de `predDemand`.

In [4]:
productList = predDemand['UPC'].unique()  # UPC pour predDemand
predDemand['PRED_SALES'] = -1  # création de la colonne PRED_SALES
for upc in productList:
    mask = predDemand['UPC'] == upc  # lignes pour upc
    X = predDemand.loc[mask, feature_list]  # caractéristiques pour upc
    y_pred = regr[upc].predict(X).clip(min=0.).round(decimals=1)  # prédictions pour upc
    predDemand.loc[mask, 'PRED_SALES'] = y_pred  # ajout des prédictions aux données

predDemand

,AVG_PRICE_VALUE,UPC,PRICE,PRICE_p2,FEATURE,DISPLAY,TPR_ONLY,RELPRICE,PRED_SALES
0,3.0,1600027528,2.5,6.25,0,0,0,0.833333,94.9
1,3.0,1600027528,3.0,9.00,0,0,0,1.000000,67.0
2,3.0,1600027528,3.5,12.25,0,0,0,1.166667,46.4
3,3.0,1600027564,2.5,6.25,0,0,0,0.833333,24.1
4,3.0,1600027564,3.0,9.00,0,0,0,1.000000,22.6
5,3.0,1600027564,3.5,12.25,0,0,0,1.166667,19.8
6,3.0,3000006340,2.5,6.25,0,0,0,0.833333,6.2
7,3.0,3000006340,3.0,9.00,0,0,0,1.000000,4.0
8,3.0,3000006340,3.5,12.25,0,0,0,1.166667,3.0
9,3.0,3800031829,2.5,6.25,0,0,0,0.833333,32.9


Nous enregistrons maintenant les ventes prévues dans un fichier csv qui sera utilisé ultérieurement dans les modèles d'optimisation.

In [5]:
# Enregistrement des prédictions dans le fichier correspondant
if SIZE == 'small':
    predDemand.to_csv('data/predictedSales_Prob1.csv', index=False)
elif SIZE == 'large':
    predDemand.to_csv('data/predictedSales_Prob2.csv', index=False)
else:
    raise ValueError("SIZE doit être défini à 'small' ou 'large'")